In [3]:
import numpy as np
import pandas as pd
from mlfinlab.online_portfolio_selection import *
#from mlfinlab.online_portfolio_selection.BAH import BAH




Support us on Patreon: https://www.patreon.com/HudsonThames

MlFinLab needs you! We need your help for us to keep on maintaining and implementing academic research based on 
financial machine learning (for open-source). In order for us to continue we need to raise $4000 of monthly donations
via Patreon - by December 2020. If we can't reach our goal, we will need to adopt more of a paid for service. We thought
that the best and least impactful course of action (should we not reach our goal) is to leave the package as open-source
but to make the documentation (ReadTheDocs) a paid for service. This is the ultimate litmus test, if the package is a 
value add, then we need the community to help us keep it going.

Our road map for 2020 is to implement the text book: Machine Learning for Asset Managers by Marcos Lopez de Prado, 
as well as a few papers from the Journal of Financial Data Science. We are hiring a full time developer for 3 months 
to help us reach our goals. The money that you

/Users/alexkwon/opt/anaconda3/envs/ht/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/alexkwon/opt/anaconda3/envs/ht/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
stock_prices = pd.read_csv('../tests/test_data/stock_prices.csv', parse_dates=True, index_col='Date')
stock_prices = stock_prices.dropna(axis=1)
asset_name = list(stock_prices.columns)

In [5]:
idx = stock_prices.index
make_array = np.array(stock_prices)
make_array

array([[ 49.2733345 ,  35.38999939, 106.63999939, ...,  77.36000061,
        101.40000153, 130.63000488],
       [ 49.71666718,  35.29000092, 107.        , ...,  77.45999908,
        101.51999664, 130.74000549],
       [ 48.22333145,  34.59999847, 106.97000122, ...,  77.55000305,
        101.65000153, 128.16999817],
       ...,
       [ 34.02000046,  23.70999908, 116.44999695, ...,  84.19000244,
        105.94000244, 176.63999939],
       [ 34.36000061,  24.04999924, 116.66999817, ...,  84.30000305,
        106.08999634, 179.        ],
       [ 34.68999863,  24.12000084, 117.05999756, ...,  84.37000275,
        106.        , 179.30999756]])

In [8]:
diff = np.array(stock_prices.pct_change().fillna(0))
diff

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00899742, -0.00282561,  0.00337585, ...,  0.00129264,
         0.00118338,  0.00084208],
       [-0.03003692, -0.01955235, -0.00028036, ...,  0.00116194,
         0.00128058, -0.01965739],
       ...,
       [ 0.02531646,  0.01368105, -0.00060075, ..., -0.0003562 ,
         0.00047222,  0.01622363],
       [ 0.00999413,  0.01433995,  0.00188923, ...,  0.00130658,
         0.00141584,  0.01336051],
       [ 0.00960413,  0.00291067,  0.00334276, ...,  0.00083036,
        -0.0008483 ,  0.00173183]])

In [12]:
weight = np.ones(23)/23
weight

array([0.04347826, 0.04347826, 0.04347826, 0.04347826, 0.04347826,
       0.04347826, 0.04347826, 0.04347826, 0.04347826, 0.04347826,
       0.04347826, 0.04347826, 0.04347826, 0.04347826, 0.04347826,
       0.04347826, 0.04347826, 0.04347826, 0.04347826, 0.04347826,
       0.04347826, 0.04347826, 0.04347826])

In [20]:
changed = np.prod(np.dot(diff[1:], weight) + np.ones(2140))
changed += np.ones(2140).reshape((2140,1))
np.prod(changed)

1.064689810916037

In [31]:
np.ones((2140,1)).shape

(2140, 1)

In [74]:
import cvxpy as cp

# Create two scalar optimization variables.
x = cp.Variable()
y = cp.Variable()

# Create two constraints.
constraints = [x + y == 1,
               x - y >= 1]

# Form objective.
obj = cp.Minimize((x - y)**2)

# Form and solve problem.
prob = cp.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x.value, y.value)

status: optimal
optimal value 1.0
optimal var 1.0 1.570086213240983e-22


In [54]:
weights = cp.Variable(23)
weights.value = np.ones(23)/23
cost = cp.prod(diff[1:]@weights + np.ones(2140))

# Optimisation objective and constraints
allocation_objective = cp.Maximize(cost)
allocation_constraints = [
    cp.sum(weights) == 1,
    weights <= 1,
    weights >= 0
]
# Define and solve the problem
problem = cp.Problem(
    objective=allocation_objective,
    constraints=allocation_constraints
)
problem.solve()
problem.value

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
Prod([[ 0.00899742 -0.00282561  0.00337585 ...  0.00129264  0.00118338
   0.00084208]
 [-0.03003692 -0.01955235 -0.00028036 ...  0.00116194  0.00128058
  -0.01965739]
 [ 0.00732708  0.00086713 -0.00018701 ...  0.00025785  0.00068863
  -0.00085824]
 ...
 [ 0.02531646  0.01368105 -0.00060075 ... -0.0003562   0.00047222
   0.01622363]
 [ 0.00999413  0.01433995  0.00188923 ...  0.00130658  0.00141584
   0.01336051]
 [ 0.00960413  0.00291067  0.00334276 ...  0.00083036 -0.0008483
   0.00173183]] * var1969555 + [1. 1. 1. ... 1. 1. 1.], None, False)

In [77]:
weights = cp.Variable(23)
# weights.value = np.ones(23)/23
cost = cp.sum(cp.log(diff[1:] * weights + np.ones(2140)))

# Optimisation objective and constraints
allocation_objective = cp.Maximize(cost)
allocation_constraints = [
    cp.sum(weights) == 1,
    weights <= 1,
    weights >= 0
]
# Define and solve the problem
problem = cp.Problem(
    objective=allocation_objective,
    constraints=allocation_constraints
)
problem.solve()
problem.value


0.5717584766291327

In [78]:
weights.value

array([6.31650159e-13, 4.57183901e-13, 1.47385062e-12, 7.73907665e-13,
       4.00243529e-13, 3.20573807e-12, 2.93296431e-13, 2.97422542e-13,
       2.46891557e-12, 1.25401503e-12, 1.54409957e-12, 1.78067518e-12,
       6.25271915e-01, 2.80894621e-12, 6.60013407e-13, 6.24704514e-13,
       2.86895944e-13, 6.17257019e-13, 7.14709998e-12, 3.74728085e-01,
       1.38310567e-12, 1.18043427e-12, 4.01980089e-12])

In [79]:
np.exp(problem.value)

1.7713792432740847